# core

> Module for the least squares model fit.

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| export
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from dataclasses import dataclass
# from scipy.stats import t
import typing


Create an instance of the class `CalibrationModel` by passing in the path to both your calibration data and your sample data. Next specify the name of your response variable and the nummber of test replicates you measured.

In [ ]:
sns.set_style("white")
sns.set_style("ticks")

In [ ]:
#| export
class CalibrationModel:

    def __init__(self, x, y, test_replicates):
      
        self.x = x
        self.y = y
        self.slope = None
        self.intercept = None
        self.r_squared = None
        self.std_err = None
        self.test_replicates = test_replicates
        self.cal_line_points = self.x.shape[0]
        self.df_resid = self.cal_line_points - 2

    def fit_ols(self):

        self.slope, self.intercept, self.r_squared, self.std_err, _ = sp.stats.linregress(self.x, self.y)
    
    def calculate_fitted_values(self):
        
        self.fitted_values = self.slope * self.x + self.intercept

   
    def inverse_prediction(self, unknown):

        if len(unknown) > 1:
            y = np.mean(unknown)
        else:
            y = unknown[0]
       
        return (y - self.intercept)/self.slope
    
    def calculate_sse(self):
        self.calculate_fitted_values()
        return np.sum((self.fitted_values - self.y) **2)
    
    def calculate_syx(self):
        return np.sqrt((self.calculate_sse())/(len(self.x)-2))

    def get_t_value(self,alpha):
        return sp.stats.t.ppf(1 - alpha/2, self.df_resid)

    def calculate_uncertainty(self):
        return self.calculate_sxhat() * self.get_t_value(0.05)
    
    def calculate_sxhat(self):
        return (self.calculate_syx() / self.slope) * np.sqrt( 1/ self.test_replicates + 1 / self.cal_line_points) 
    
    def fit_model(self):
        self.fit_ols()
        self.calculate_uncertainty()
        self.tabulate_results()

    def tabulate_results(self):
        print(f"Calibration curve")
        print(f"R2 = {self.r_squared}")
        print(f"Slope = {self.slope}")
        print(f"Intercept = {self.intercept}")
        print(f"Uncertainity = {self.calculate_uncertainty()}")
        # print(f"Prediction = {self.inverse_prediction()}")
    


In [ ]:
#| export
# def plot_calibration_curve(model, df, response_variable):

#         fig, ax = plt.subplots(1, 1, figsize=(5, 5))
#         sns.regplot(x="concentration", y=response_variable, data=df)
#         # ax.annotate(f"Predicted value = {model.inverse_prediction():.2f}", xy=(0.5, 0.9), xycoords='axes fraction', fontsize=9, ha='center', va='center')
#         # ax.axvline(x=model.inverse_prediction(), color='red', linestyle='--')
#         # ax.axvline(x=model.inverse_prediction() + model.calculate_uncertainty(), color='blue', linestyle='--')
#         # ax.axvline(x=model.inverse_prediction() - model.calculate_uncertainty(), color='blue', linestyle='--')
#         # ax.set_title('Calibration Curve')
#         # ax.set_xlabel('Concentration (v/v%)')
#         # ax.set_ylabel('Peak Value')
#         # ax.annotate(f"R-squared = {model.rsquared:.3f}", xy=(0.5, 0.8), xycoords='axes fraction', fontsize=9, ha='center', va='center')
#         # ax.annotate(f"Regression formula: y = {model.get_params()[0]:.3f} * x + {model.get_params()[1]:.3f}", xy=(0.5, 0.7), xycoords='axes fraction', fontsize=9, ha='center', va='center')

#         plt.tight_layout()
#         plt.show()

## Tests

In [ ]:
def generate_test_data(slope, intercept):
        x = np.linspace(1, 10, num=5)
        y = intercept + x * slope
        df = pd.DataFrame({'concentration': x, "abs": y})
        return df
def generate_sample_data():
    x = np.array(['unknown1', 'unknown2'])
    y = np.array([13.75, 20.50])
    df = pd.DataFrame({'sample': x, "abs": y})
    df = df.set_index('sample')
    return df


# test_data = generate_test_data(3, 4)
# sample_data = generate_sample_data()

test_data = pd.DataFrame({'concentration': [0.2, 0.05, 0.1, 0.8, 0.6, 0.4], "abs": [0.221, 0.057, 0.119, 0.73, 0.599, 0.383]})
sample_data = pd.DataFrame({'unknown': [0.490, 0.471, 0.484, 0.473, 0.479, 0.492]})


In [ ]:
cal = CalibrationModel(x=test_data['concentration'], y=test_data['abs'], test_replicates=6)
cal.fit_ols()
print(cal.calculate_uncertainty())
print(cal.slope)
print(cal.intercept)
print(cal.r_squared)
print(cal.calculate_syx())

0.036768287028466705
0.9044109330819979
0.027419415645617395
0.9976282521058687
0.020744870935306556


In [ ]:
cal.fit_model()

Calibration curve
R2 = 0.9976282521058687
Slope = 0.9044109330819979
Intercept = 0.027419415645617395
Uncertainity = 0.036768287028466705


In [ ]:
cal.inverse_prediction(sample_data['unknown'])

0.5020733029033536

In [ ]:
cal.tabulate_results()

Calibration curve
R2 = 0.9976282521058687
Slope = 0.9044109330819979
Intercept = 0.027419415645617395
Uncertainity = 0.036768287028466705


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()